In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Import the datetime module from the datetime library.
from datetime import datetime

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [4]:
# Use the tuple() function to display the latitude and longitude combinations.
for coordinate in coordinates:
    coordinate = (coordinate[0], coordinate[1])

In [5]:
from citipy import citipy
for coordinate in coordinates:
    city_name = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    city_countryCode = citipy.nearest_city(coordinate[0], coordinate[1]).country_code

In [6]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)
#print(cities)

763

In [7]:
# Create an empty list to hold the weather data.
city_data = []

# Create counters.
record_count = 1
set_count = 1

In [8]:
# Import the requests library.
import requests

# Import the API key.
from config import weather_api_key

In [9]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [10]:
# Loop through all the cities in our list.
for i in range(len(cities)):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + cities[i]
    

In [11]:
city_weather = requests.get(city_url).json()
#city_weather

In [12]:
city_des = city_weather["weather"][0]["description"]
city_des

'clear sky'

In [13]:
# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_des = city_weather["weather"][0]["description"]
        city_country = city_weather["sys"]["country"]
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_des,
                          "Country": city_country})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

    


-----------------------------
Data Retrieval Complete      
-----------------------------


In [14]:
len(city_data)

705

In [15]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Country
0,Kapaa,22.0752,-159.3190,80.19,71,72,3.00,light rain,US
1,Mul,20.0667,79.6667,82.81,73,100,5.64,overcast clouds,IN
2,Albany,42.6001,-73.9662,81.30,75,99,1.72,overcast clouds,US
3,Ponta Do Sol,32.6667,-17.1000,72.72,79,7,4.18,clear sky,PT
4,Hobart,-42.8794,147.3294,48.22,78,40,3.00,light rain,AU
5,Lebu,-37.6167,-73.6500,54.01,85,97,14.29,overcast clouds,CL
6,Qaanaaq,77.4840,-69.3632,38.59,87,90,2.10,overcast clouds,GL
7,Ushuaia,-54.8000,-68.3000,42.46,70,40,4.61,scattered clouds,AR
8,Mataura,-46.1927,168.8643,36.48,92,47,3.20,scattered clouds,NZ
9,Maningrida,-12.0667,134.2667,63.32,92,72,8.37,broken clouds,AU


In [16]:
new_column_order = ["City", "Country", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Current Description"]
city_data_df = city_data_df[new_column_order]
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Kapaa,US,22.0752,-159.3190,80.19,71,72,3.00,light rain
1,Mul,IN,20.0667,79.6667,82.81,73,100,5.64,overcast clouds
2,Albany,US,42.6001,-73.9662,81.30,75,99,1.72,overcast clouds
3,Ponta Do Sol,PT,32.6667,-17.1000,72.72,79,7,4.18,clear sky
4,Hobart,AU,-42.8794,147.3294,48.22,78,40,3.00,light rain
5,Lebu,CL,-37.6167,-73.6500,54.01,85,97,14.29,overcast clouds
6,Qaanaaq,GL,77.4840,-69.3632,38.59,87,90,2.10,overcast clouds
7,Ushuaia,AR,-54.8000,-68.3000,42.46,70,40,4.61,scattered clouds
8,Mataura,NZ,-46.1927,168.8643,36.48,92,47,3.20,scattered clouds
9,Maningrida,AU,-12.0667,134.2667,63.32,92,72,8.37,broken clouds


In [17]:
# Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")